In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [2]:
epochs = 10
range_min = 0
range_max = 999
batch_size = 4 
num_classes = 2
num_train_samples = 100
num_test_samples = 100
num_digits = 3

In [4]:
temp = np.random.randint(low = 0,high = range_max+1,size = num_train_samples)
x_train = np.empty((num_train_samples,3))
x_train[:,0] = temp
x_train[:num_train_samples // 2,1] = 1  
x_train[num_train_samples // 2:,1] = -1 
x_train[:,2] = num_digits
np.random.shuffle(x_train)
y_train = x_train[:,0] * x_train[:,1]
print(x_train.shape)
print(y_train.shape)

(100, 3)
(100,)


In [5]:
train_data = torch.from_numpy(x_train).type(torch.FloatTensor)
train_labels = torch.from_numpy(y_train).type(torch.LongTensor)
# test_data = torch.from_numpy(x_test).type(torch.FloatTensor)
# test_labels = torch.from_numpy(y_test).type(torch.LongTensor)

In [6]:
# This model subtracts two numbers
class Net_sub(nn.Module):
    def __init__(self):
        super(Net_sub,self).__init__()
        self.fc1 = nn.Linear(2,1)
        self.fc1.weight.data = torch.tensor([[1,-1]]).type(torch.FloatTensor)
        self.fc1.bias.data = torch.tensor([0]).type(torch.FloatTensor)
    
    def forward(self,x):
        x = self.fc1(x)
        return x
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)
    
model_sub = torch.load('model_sub')

In [7]:
# This model will separate a number into digits
# input: 2 inputs - (number,num_digits)
# output: separated digits (num_digits,)  

class Net_separate(nn.Module):
    def __init__(self,model_sub):
        super(Net_separate,self).__init__()
        
        self.divide_by_10 = nn.Linear(1,1,bias=False)
        self.divide_by_10.weight.data = torch.tensor([[0.1]]).type(torch.FloatTensor)

        self.multiply_by_10 = nn.Linear(1,1,bias=False)
        self.multiply_by_10.weight.data = torch.tensor([[10]]).type(torch.FloatTensor)
        
        self.model_sub = model_sub
    
    def forward(self,x):
        input_number = x[:,0].unsqueeze(-1)
        num_digits = x[:,1].unsqueeze(-1)
        concat = []
        data = input_number
        for i in range(num_digits[0].type(torch.IntTensor)):
            temp_data = self.divide_by_10(data).floor()
            temp = self.multiply_by_10(temp_data)
            concat_temp = torch.cat((data,temp),1)
            digit = self.model_sub.predict(concat_temp)
            
            concat.insert(0,digit)
            data = temp_data
        
        out = torch.cat(concat,1)
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)
    
model_separate = torch.load('model_separate')    

In [9]:
# this model mulltiplies a single digit number with a sign (1,-1)
# input: 2 numbers (2,)
# output: single number 
class Net_mul_sign_0_9(nn.Module):
    def __init__(self):
        super(Net_mul_sign_0_9,self).__init__()
        self.fc1 = nn.Linear(2,10)
        self.fc2 = nn.Linear(10,1)
    
    def forward(self,x):
        x = F.softsign(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def predict(self,x):
        out = self.forward(x)
        return torch.round(out).type(torch.FloatTensor)

model_mul_sign_0_9 = torch.load('model_mul_sign_0_9')

In [42]:
# this model mulltiplies a number with a sign (1,-1)
# input: 3 numbers (3,) - [number,sign,num_digits]
# output: single number 
class Net_mul_sign_abs(nn.Module):
    def __init__(self,model_separate,model_mul_sign_0_9):
        super(Net_mul_sign_abs,self).__init__()
        self.model_separate = model_separate
        self.model_mul_sign_0_9 = model_mul_sign_0_9
        self.multiply_2 = nn.Linear(1,1,bias = False)
        self.multiply_2.weight.data = torch.tensor([[2.0]])
        self.layers = []
    
    def forward(self,x):
        number = x[:,0].unsqueeze(-1)
        sign = x[:,1].unsqueeze(-1)
        num_digits = x[:,2].unsqueeze(-1)
        
        output_digits = self.multiply_2(num_digits)
        concat = torch.cat((number,output_digits),1)
        sep = self.model_separate.predict(concat)
        
        products = []
        i = 0
        j = sep.shape[-1]
        for n in range(j,0,-1):
            temp = sep[:,n-1:n]
            concat = torch.cat((temp,sign),1)
            prod = self.model_mul_sign_0_9.predict(concat)
            self.layers.append(nn.Linear(1,1,bias=False))
            self.layers[-1].weight.data = torch.tensor([[10**i]]).type(torch.FloatTensor)
            prod = self.layers[-1](prod)
            products.append(prod)
            i+=1
        
        concat = torch.cat(products,1)
        self.layers.append(nn.Linear(1,1,bias=False))
        self.layers[-1].weight.data = np.repeat(torch.tensor([[1.0]]),len(products),axis = -1)
        out = self.layers[-1](concat)
        
        return out
    
    def predict(self,x):
        out = self.forward(x)
        return out.type(torch.FloatTensor)

In [43]:
net = Net_mul_sign_abs(model_separate,model_mul_sign_0_9)

In [46]:
# print(train_data[0:10])
print(train_labels[0:1])
net.forward(train_data[0:1])

tensor([-308])


tensor([[-308.]], grad_fn=<MmBackward>)

In [47]:
torch.save(net,'model_mul_sign_abs')

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_mul_sign_abs. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_separate. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_sub. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_mul_sign_0_9. It won't be checked for correctness upo